<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-pre-processing" data-toc-modified-id="Data-pre-processing-1">Data pre-processing</a></span></li><li><span><a href="#Gradient-Boosted-Trees" data-toc-modified-id="Gradient-Boosted-Trees-2">Gradient Boosted Trees</a></span><ul class="toc-item"><li><span><a href="#HistGradientBoostingClassifier" data-toc-modified-id="HistGradientBoostingClassifier-2.1">HistGradientBoostingClassifier</a></span><ul class="toc-item"><li><span><a href="#Initial-Classifiers" data-toc-modified-id="Initial-Classifiers-2.1.1">Initial Classifiers</a></span></li><li><span><a href="#Hyperparameter-tuning" data-toc-modified-id="Hyperparameter-tuning-2.1.2">Hyperparameter tuning</a></span><ul class="toc-item"><li><span><a href="#GridSearchCV" data-toc-modified-id="GridSearchCV-2.1.2.1">GridSearchCV</a></span></li><li><span><a href="#RandomizedSearchCV" data-toc-modified-id="RandomizedSearchCV-2.1.2.2">RandomizedSearchCV</a></span></li></ul></li><li><span><a href="#Including-the-V***-columns" data-toc-modified-id="Including-the-V***-columns-2.1.3">Including the V*** columns</a></span></li></ul></li><li><span><a href="#GradientBoostingClassifier" data-toc-modified-id="GradientBoostingClassifier-2.2">GradientBoostingClassifier</a></span><ul class="toc-item"><li><span><a href="#Defaults" data-toc-modified-id="Defaults-2.2.1">Defaults</a></span></li><li><span><a href="#Hyperparameter-Tuning" data-toc-modified-id="Hyperparameter-Tuning-2.2.2">Hyperparameter Tuning</a></span></li></ul></li></ul></li></ul></div>

# Data pre-processing

In [2]:
from fastai.tabular.all import *
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 200
pd.options.display.max_columns = 125

In [7]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to_ss.pkl').load()

In [8]:
to["card1"].dtype

dtype('int16')

In [9]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

for cat in cats:
    to[cat] = to[cat].astype('category')

In [10]:
to["card1"].dtype

CategoricalDtype(categories=[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
                  ...
                  5634, 5635, 5636, 5637, 5638, 5639, 5640, 5641, 5642, 5643],
                 ordered=False)

In [11]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [12]:
xs.shape

(48357, 122)

# Gradient Boosted Trees

## HistGradientBoostingClassifier

In [112]:
'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

In [113]:
'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

In [114]:
'BOTH: Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

'Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

In [115]:
'LGBM: This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

'This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

In [116]:
'LGBM: Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

'Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \\gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

HistGradientBoostingClassifier is based on Microsofts LightGBM.

### Initial Classifiers 

In [3]:
def m_roc(y, xs):
    preds = clf.predict_proba(xs)
    preds_isFraud = preds[:,1]
    return roc_auc_score(y, preds_isFraud)

The max_iter parameter is increased to 1000 from its default value of 100.

In [22]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc')
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.286 s
Binning 0.005 GB of validation data: 0.021 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.83480, val score: 0.78900, in 0.062s
[2/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.86395, val score: 0.81880, in 0.073s
[3/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.86796, val score: 0.81789, in 0.059s
[4/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.89040, val score: 0.85394, in 0.064s
[5/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.89595, val score: 0.85530, in 0.066s
[6/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.90007, val score: 0.85618, in 0.065s
[7/1000] 1 tree, 31 leaves, max depth = 13, train score: 0.90363, val score: 0.85351, in 0.066s
[8/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.90634, val score: 0.86130, in 0.066s
[9/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.91021, val score: 0.86198, in 0.071s
[10/1000] 1 tree,

(0.9658515572705477, 0.8836308129568515)

The algorithm may have stopped too early, so let's increase the threshold for early stopping by decreasing tol by a factor of 10 to e-8 from its default value e-7.

In [17]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                     max_iter=1000, scoring='roc_auc', tol=1e-8)
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.271 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.78101, val score: 0.79729, in 0.058s
[2/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.81562, val score: 0.84685, in 0.083s
[3/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.84869, val score: 0.87510, in 0.047s
[4/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.85485, val score: 0.88094, in 0.047s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86568, val score: 0.88349, in 0.051s
[6/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87742, val score: 0.88483, in 0.044s
[7/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.88160, val score: 0.88449, in 0.047s
[8/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.88430, val score: 0.88644, in 0.058s
[9/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.88651, val score: 0.88777, in 0.058s
[10/1000] 1 tree, 31 l

(0.9736014537452244, 0.8878831140620626)

In [41]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.277 s
Binning 0.005 GB of validation data: 0.020 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.86889, val score: 0.82889, in 0.082s
[2/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87925, val score: 0.82951, in 0.083s
[3/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.88058, val score: 0.83309, in 0.082s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88378, val score: 0.83707, in 0.160s
[5/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.88396, val score: 0.83800, in 0.085s
[6/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.88424, val score: 0.83771, in 0.088s
[7/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88430, val score: 0.83762, in 0.087s
[8/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.88433, val score: 0.83813, in 0.087s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88446, val score: 0.83836, in 0.090s
[10/1000] 1 tree,

[85/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93967, val score: 0.88073, in 0.120s
[86/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.94014, val score: 0.88105, in 0.122s
[87/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94048, val score: 0.88154, in 0.119s
[88/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.94068, val score: 0.88203, in 0.118s
[89/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.94092, val score: 0.88201, in 0.119s
[90/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94134, val score: 0.88236, in 0.202s
[91/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.94175, val score: 0.88235, in 0.111s
[92/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.94205, val score: 0.88261, in 0.117s
[93/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94233, val score: 0.88285, in 0.116s
[94/1000] 1 tree, 44 leaves, max depth = 8, train score: 0.94257, val score: 0.88302, in 0.117s
[95/1000] 1 tree, 44 leaves, ma

[169/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96550, val score: 0.89445, in 0.144s
[170/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96576, val score: 0.89463, in 0.141s
[171/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.96603, val score: 0.89479, in 0.138s
[172/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.96607, val score: 0.89486, in 0.135s
[173/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96627, val score: 0.89500, in 0.135s
[174/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.96658, val score: 0.89515, in 0.137s
[175/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96689, val score: 0.89502, in 0.138s
[176/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.96698, val score: 0.89483, in 0.135s
[177/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96725, val score: 0.89498, in 0.138s
[178/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96737, val score: 0.89498, in 0.225s
[179/1000] 1 tree, 4

CPU times: user 3min 36s, sys: 37.1 s, total: 4min 13s
Wall time: 36.4 s


(0.9697907275897766, 0.8931305761072392)

In [4]:
xs_imp = (path/'xs_imp.pkl').load()
valid_xs_imp = (path/'valid_xs_imp.pkl').load()

In [14]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs_imp, y)
m_roc(y, xs_imp), m_roc(valid_y, valid_xs_imp)

Binning 0.012 GB of training data: 0.120 s
Binning 0.001 GB of validation data: 0.006 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84702, val score: 0.81191, in 0.050s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.85147, val score: 0.82025, in 0.050s
[3/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.85412, val score: 0.82111, in 0.051s
[4/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86112, val score: 0.83071, in 0.049s
[5/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.86121, val score: 0.83201, in 0.048s
[6/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.86392, val score: 0.83442, in 0.064s
[7/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.86470, val score: 0.83532, in 0.042s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86587, val score: 0.83555, in 0.050s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.86615, val score: 0.83657, in 0.057s
[10/1000] 1 tree

[85/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.91802, val score: 0.87532, in 0.056s
[86/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.91856, val score: 0.87588, in 0.053s
[87/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.91921, val score: 0.87633, in 0.061s
[88/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.91978, val score: 0.87620, in 0.054s
[89/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.92020, val score: 0.87660, in 0.052s
[90/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.92077, val score: 0.87683, in 0.052s
[91/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92135, val score: 0.87690, in 0.054s
[92/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.92170, val score: 0.87721, in 0.082s
[93/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92207, val score: 0.87749, in 0.048s
[94/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.92261, val score: 0.87733, in 0.060s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.95712, val score: 0.88853, in 0.081s
[170/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.95739, val score: 0.88869, in 0.080s
[171/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.95761, val score: 0.88912, in 0.085s
[172/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.95779, val score: 0.88919, in 0.080s
[173/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.95795, val score: 0.88926, in 0.083s
[174/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.95814, val score: 0.88936, in 0.082s
[175/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.95837, val score: 0.88932, in 0.083s
[176/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.95857, val score: 0.88931, in 0.092s
[177/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.95867, val score: 0.88969, in 0.087s
[178/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.95886, val score: 0.88960, in 0.086s
[179/1000] 1 tree, 4

[253/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.97196, val score: 0.89860, in 0.109s
[254/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.97209, val score: 0.89869, in 0.111s
[255/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.97219, val score: 0.89869, in 0.142s
[256/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.97227, val score: 0.89888, in 0.111s
[257/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.97241, val score: 0.89888, in 0.114s
[258/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.97251, val score: 0.89919, in 0.115s
[259/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.97257, val score: 0.89920, in 0.118s
[260/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.97264, val score: 0.89933, in 0.116s
[261/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.97282, val score: 0.89929, in 0.114s
[262/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.97297, val score: 0.89930, in 0.121s
[263/1000] 1 tree, 44

[337/1000] 1 tree, 44 leaves, max depth = 22, train score: 0.98030, val score: 0.90378, in 0.147s
[338/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.98043, val score: 0.90372, in 0.141s
[339/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98051, val score: 0.90372, in 0.144s
[340/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98059, val score: 0.90356, in 0.134s
[341/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98067, val score: 0.90360, in 0.144s
[342/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98082, val score: 0.90374, in 0.140s
[343/1000] 1 tree, 44 leaves, max depth = 22, train score: 0.98089, val score: 0.90360, in 0.168s
[344/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98093, val score: 0.90347, in 0.140s
[345/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.98102, val score: 0.90354, in 0.151s
[346/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.98105, val score: 0.90346, in 0.169s
Fit 346 trees in 32.

(0.9746698614946053, 0.8975705249163175)

### Hyperparameter tuning

Hyperparameters were tuned interatively. Using both GridSearchCV and RandomizedGridSearchCV. RandomizedGridSearchCV was used for all 5 sets of parameters, than the final parameter set was double checked using GridSearchCV. Only the final results are displayed in the notenbook.

In [16]:
parameters1 = {
 'learning_rate': [0.01, 0.1, 10],
 'max_depth': [None, 25],
 'l2_regularization': [0.0, 3.0],
 'max_leaf_nodes': [10, 100, None],
 'min_samples_leaf': [1, 10, 100],
 'random_state': [33],
 }

In [50]:
parameters2 = {
 'learning_rate': [0.1, 0.2, 0.3],
 'max_depth': [25, 50],
 'l2_regularization': [1.5, 3.0],
 'max_leaf_nodes': [50, 100],
 'min_samples_leaf': [1, 5],
 'random_state': [33],
 }

In [56]:
parameters3 = {
 'learning_rate': [0.005, 0.05, 0.1],
 'max_depth': [15, 25],
 'l2_regularization': [2.0, 3.0],
 'max_leaf_nodes': [20, 35, 50],
 'min_samples_leaf': [1, 3],
 'random_state': [33],
 }

In [62]:
parameters4 = {
 'learning_rate': [0.1, 0.14],
 'max_depth': [10, 15],
 'l2_regularization': [2.5, 3.0],
 'max_leaf_nodes': [40, 50],
 'min_samples_leaf': [1, 2],
 'random_state': [33],
 }

In [64]:
parameters5 = {
 'learning_rate': [0.1],
 'max_depth': [15],
 'l2_regularization': [2.8, 3.0],
 'max_leaf_nodes': [45, 50],
 'min_samples_leaf': [2],
 'random_state': [33],
 }

#### GridSearchCV

In [17]:
clf = GridSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters1, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [18]:
# This had taken over 2 hrs before I gave up on it.
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/fastaiV2/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/fastaiV2/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 567, in run
    self.flag_executor_shutting_down()
  File "/opt/anaconda3/envs/fastaiV2/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 756, in flag_executor_shutting_down
    self.kill_workers()
  File "/opt/anaconda3/envs/fastaiV2/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 766, in kill_workers
    recursive_terminate(p)
  File "/opt/anaconda3/envs/fastaiV2/lib/python3.8/site-packages/joblib/externals/loky/backend/utils.py", line 28, in recursive_terminate
    _recursive_terminate_without_psutil(process)
  File "/opt/anaconda3/envs/fastaiV2/lib/python3.8/site-packages/joblib/externals/loky/backend/utils.py", line 53, in _recursive_termina

KeyboardInterrupt: 

In [42]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.273 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.83422, val score: 0.83526, in 0.076s
[2/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84166, val score: 0.84216, in 0.081s
[3/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.84365, val score: 0.84422, in 0.081s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84635, val score: 0.84409, in 0.120s
[5/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85918, val score: 0.85764, in 0.079s
[6/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.85980, val score: 0.85770, in 0.122s
[7/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85984, val score: 0.85860, in 0.120s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86009, val score: 0.85848, in 0.130s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.86031, val score: 0.85919, in 0.116s
[10/1000] 1 tree,

[85/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92742, val score: 0.90149, in 0.116s
[86/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92801, val score: 0.90142, in 0.117s
[87/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.92856, val score: 0.90216, in 0.111s
[88/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92926, val score: 0.90224, in 0.111s
[89/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.92989, val score: 0.90258, in 0.107s
[90/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.93051, val score: 0.90300, in 0.181s
[91/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93145, val score: 0.90335, in 0.106s
[92/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.93208, val score: 0.90435, in 0.272s
[93/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93265, val score: 0.90504, in 0.110s
[94/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93347, val score: 0.90482, in 0.107s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 20, train score: 0.96254, val score: 0.91969, in 0.204s
[170/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.96271, val score: 0.91994, in 0.156s
[171/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96299, val score: 0.91994, in 0.145s
[172/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96309, val score: 0.92024, in 0.142s
[173/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96335, val score: 0.92045, in 0.171s
[174/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96377, val score: 0.92087, in 0.175s
[175/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.96408, val score: 0.92078, in 0.150s
[176/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96426, val score: 0.92085, in 0.141s
[177/1000] 1 tree, 44 leaves, max depth = 21, train score: 0.96446, val score: 0.92099, in 0.145s
[178/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96457, val score: 0.92096, in 0.211s
[179/1000] 1 tree, 4

(0.9659117013340929, 0.8577956250355605)

#### RandomizedSearchCV

In [45]:
HistGradientBoostingClassifier?

In [47]:
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters1, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [48]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 22s, sys: 31.6 s, total: 1min 53s
Wall time: 5min 55s


(HistGradientBoostingClassifier(l2_regularization=3.0,
                                loss='binary_crossentropy', max_depth=25,
                                max_iter=1000, max_leaf_nodes=100,
                                min_samples_leaf=1, random_state=33,
                                scoring='roc_auc', tol=1e-08),
 0.8994341794744841,
 {'random_state': 33,
  'min_samples_leaf': 1,
  'max_leaf_nodes': 100,
  'max_depth': 25,
  'learning_rate': 0.1,
  'l2_regularization': 3.0})

In [52]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=3.0,
                             learning_rate=0.1, max_depth=25, max_iter=1000, max_leaf_nodes=100,
                             min_samples_leaf=1,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.267 s
Binning 0.005 GB of validation data: 0.020 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 100 leaves, max depth = 17, train score: 0.83818, val score: 0.85907, in 0.103s
[2/1000] 1 tree, 100 leaves, max depth = 12, train score: 0.86476, val score: 0.86264, in 0.105s
[3/1000] 1 tree, 100 leaves, max depth = 15, train score: 0.87483, val score: 0.86511, in 0.112s
[4/1000] 1 tree, 100 leaves, max depth = 13, train score: 0.88018, val score: 0.87326, in 0.154s
[5/1000] 1 tree, 100 leaves, max depth = 15, train score: 0.88565, val score: 0.87560, in 0.112s
[6/1000] 1 tree, 100 leaves, max depth = 15, train score: 0.89286, val score: 0.88103, in 0.116s
[7/1000] 1 tree, 100 leaves, max depth = 15, train score: 0.89638, val score: 0.88804, in 0.122s
[8/1000] 1 tree, 100 leaves, max depth = 14, train score: 0.89996, val score: 0.88886, in 0.116s
[9/1000] 1 tree, 100 leaves, max depth = 13, train score: 0.90441, val score: 0.89151, in 0.166s
[10/10

(0.9902917443033911, 0.897707662150385)

In [53]:
%%time
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters2, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 43.7 s, sys: 12.6 s, total: 56.4 s
Wall time: 4min 9s


(HistGradientBoostingClassifier(l2_regularization=3.0,
                                loss='binary_crossentropy', max_depth=25,
                                max_iter=1000, max_leaf_nodes=50,
                                min_samples_leaf=1, random_state=33,
                                scoring='roc_auc', tol=1e-08),
 0.9029214381739428,
 {'random_state': 33,
  'min_samples_leaf': 1,
  'max_leaf_nodes': 50,
  'max_depth': 25,
  'learning_rate': 0.1,
  'l2_regularization': 3.0})

In [54]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=3.0,
                             learning_rate=0.1, max_depth=25, max_iter=1000, max_leaf_nodes=50,
                             min_samples_leaf=1,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.266 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 50 leaves, max depth = 10, train score: 0.87151, val score: 0.83116, in 0.069s
[2/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.87735, val score: 0.84413, in 0.070s
[3/1000] 1 tree, 50 leaves, max depth = 13, train score: 0.87760, val score: 0.85392, in 0.079s
[4/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.87907, val score: 0.85628, in 0.069s
[5/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.88685, val score: 0.86472, in 0.100s
[6/1000] 1 tree, 50 leaves, max depth = 10, train score: 0.89489, val score: 0.86919, in 0.065s
[7/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.89657, val score: 0.87336, in 0.068s
[8/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.90299, val score: 0.88313, in 0.069s
[9/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.90551, val score: 0.88614, in 0.067s
[10/1000] 1 tre

[85/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.98479, val score: 0.92269, in 0.092s
[86/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98516, val score: 0.92214, in 0.095s
[87/1000] 1 tree, 50 leaves, max depth = 20, train score: 0.98542, val score: 0.92222, in 0.100s
[88/1000] 1 tree, 50 leaves, max depth = 16, train score: 0.98552, val score: 0.92233, in 0.091s
[89/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.98581, val score: 0.92187, in 0.094s
[90/1000] 1 tree, 50 leaves, max depth = 13, train score: 0.98629, val score: 0.92166, in 0.098s
[91/1000] 1 tree, 50 leaves, max depth = 14, train score: 0.98666, val score: 0.92160, in 0.104s
[92/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.98687, val score: 0.92169, in 0.161s
[93/1000] 1 tree, 50 leaves, max depth = 14, train score: 0.98729, val score: 0.92132, in 0.092s
Fit 93 trees in 8.831 s, (4650 total leaves)
Time spent computing histograms: 3.472s
Time spent finding best splits:  1.028s
Ti

(0.9799191024791805, 0.9001188173745548)

In [57]:
%%time
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters3, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 2min 20s, sys: 31.1 s, total: 2min 51s
Wall time: 5min 8s


(HistGradientBoostingClassifier(l2_regularization=3.0,
                                loss='binary_crossentropy', max_depth=15,
                                max_iter=1000, max_leaf_nodes=50,
                                min_samples_leaf=1, random_state=33,
                                scoring='roc_auc', tol=1e-08),
 0.9018079437903692,
 {'random_state': 33,
  'min_samples_leaf': 1,
  'max_leaf_nodes': 50,
  'max_depth': 15,
  'learning_rate': 0.1,
  'l2_regularization': 3.0})

In [59]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=3.0,
                             learning_rate=0.1, max_depth=15, max_iter=1000, max_leaf_nodes=50,
                             min_samples_leaf=1,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.262 s
Binning 0.005 GB of validation data: 0.018 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.81823, val score: 0.79194, in 0.066s
[2/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.85544, val score: 0.83462, in 0.069s
[3/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.86601, val score: 0.84264, in 0.071s
[4/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.87332, val score: 0.84650, in 0.068s
[5/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.87767, val score: 0.84732, in 0.066s
[6/1000] 1 tree, 50 leaves, max depth = 10, train score: 0.88572, val score: 0.85011, in 0.068s
[7/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.88758, val score: 0.85344, in 0.132s
[8/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.89330, val score: 0.85601, in 0.095s
[9/1000] 1 tree, 50 leaves, max depth = 10, train score: 0.89490, val score: 0.85643, in 0.086s
[10/1000] 1 tre

[85/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98365, val score: 0.89363, in 0.159s
[86/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.98390, val score: 0.89314, in 0.103s
[87/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98409, val score: 0.89362, in 0.105s
[88/1000] 1 tree, 50 leaves, max depth = 14, train score: 0.98467, val score: 0.89291, in 0.110s
[89/1000] 1 tree, 50 leaves, max depth = 14, train score: 0.98498, val score: 0.89257, in 0.118s
[90/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98547, val score: 0.89284, in 0.142s
[91/1000] 1 tree, 50 leaves, max depth = 13, train score: 0.98557, val score: 0.89280, in 0.157s
[92/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98599, val score: 0.89340, in 0.186s
[93/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98630, val score: 0.89377, in 0.150s
[94/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.98652, val score: 0.89337, in 0.133s
[95/1000] 1 tree, 50 leaves, m

(0.9838685019824682, 0.8929366473086819)

In [63]:
%%time
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters4, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 2min 4s, sys: 29 s, total: 2min 33s
Wall time: 3min 57s


(HistGradientBoostingClassifier(l2_regularization=3.0,
                                loss='binary_crossentropy', max_depth=15,
                                max_iter=1000, max_leaf_nodes=50,
                                min_samples_leaf=2, random_state=33,
                                scoring='roc_auc', tol=1e-08),
 0.9018079437903692,
 {'random_state': 33,
  'min_samples_leaf': 2,
  'max_leaf_nodes': 50,
  'max_depth': 15,
  'learning_rate': 0.1,
  'l2_regularization': 3.0})

In [65]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=3.0,
                             learning_rate=0.1, max_depth=15, max_iter=1000, max_leaf_nodes=50,
                             min_samples_leaf=2,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.273 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.83740, val score: 0.85548, in 0.112s
[2/1000] 1 tree, 50 leaves, max depth = 10, train score: 0.84226, val score: 0.86478, in 0.062s
[3/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.85411, val score: 0.86694, in 0.064s
[4/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.86896, val score: 0.86991, in 0.067s
[5/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.87160, val score: 0.87523, in 0.078s
[6/1000] 1 tree, 50 leaves, max depth = 10, train score: 0.88209, val score: 0.87788, in 0.065s
[7/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.88565, val score: 0.87769, in 0.065s
[8/1000] 1 tree, 50 leaves, max depth = 12, train score: 0.89207, val score: 0.87742, in 0.070s
[9/1000] 1 tree, 50 leaves, max depth = 9, train score: 0.89357, val score: 0.87849, in 0.067s
[10/1000] 1 tree

[85/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98453, val score: 0.90502, in 0.099s
[86/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98483, val score: 0.90511, in 0.100s
[87/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98518, val score: 0.90529, in 0.094s
[88/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98566, val score: 0.90542, in 0.090s
[89/1000] 1 tree, 50 leaves, max depth = 10, train score: 0.98604, val score: 0.90542, in 0.157s
[90/1000] 1 tree, 50 leaves, max depth = 13, train score: 0.98621, val score: 0.90549, in 0.275s
[91/1000] 1 tree, 50 leaves, max depth = 15, train score: 0.98631, val score: 0.90543, in 0.097s
[92/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.98655, val score: 0.90521, in 0.097s
[93/1000] 1 tree, 50 leaves, max depth = 11, train score: 0.98687, val score: 0.90488, in 0.097s
[94/1000] 1 tree, 50 leaves, max depth = 13, train score: 0.98703, val score: 0.90479, in 0.098s
[95/1000] 1 tree, 50 leaves, m

(0.9805941912488121, 0.8950722195033224)

In [66]:
%%time
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

/opt/anaconda3/envs/fastaiV2/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: user 1min 44s, sys: 24.7 s, total: 2min 9s
Wall time: 1min 40s


(HistGradientBoostingClassifier(l2_regularization=3.0,
                                loss='binary_crossentropy', max_depth=15,
                                max_iter=1000, max_leaf_nodes=50,
                                min_samples_leaf=2, random_state=33,
                                scoring='roc_auc', tol=1e-08),
 0.9018079437903692,
 {'random_state': 33,
  'min_samples_leaf': 2,
  'max_leaf_nodes': 50,
  'max_depth': 15,
  'learning_rate': 0.1,
  'l2_regularization': 3.0})

In [ ]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=3.0,
                             learning_rate=0.1, max_depth=15, max_iter=1000, max_leaf_nodes=50,
                             min_samples_leaf=2,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

### Including the V*** columns

Normalized data helped this model. 

In [15]:
to = (path/'to_nn_full.pkl').load()

xs, y             = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [16]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

for cat in cats:
    to[cat] = to[cat].astype('category')

In [17]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=27, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.242 GB of training data: 1.220 s
Binning 0.027 GB of validation data: 0.043 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85319, val score: 0.83631, in 0.171s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87135, val score: 0.85190, in 0.172s
[3/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.87432, val score: 0.85400, in 0.178s
[4/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.87636, val score: 0.85526, in 0.181s
[5/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.87834, val score: 0.85747, in 0.185s
[6/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88077, val score: 0.86165, in 0.189s
[7/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.88131, val score: 0.86212, in 0.189s
[8/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.88220, val score: 0.86338, in 0.242s
[9/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.88310, val score: 0.86382, in 0.391s
[10/1000] 1 tree

[85/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.92933, val score: 0.89229, in 0.538s
[86/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92998, val score: 0.89222, in 0.552s
[87/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93055, val score: 0.89241, in 0.893s
[88/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93083, val score: 0.89268, in 0.300s
[89/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93104, val score: 0.89271, in 0.329s
[90/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.93151, val score: 0.89299, in 0.307s
[91/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.93188, val score: 0.89320, in 0.297s
[92/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.93255, val score: 0.89318, in 0.309s
[93/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.93322, val score: 0.89329, in 0.366s
[94/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.93374, val score: 0.89327, in 0.495s
[95/1000] 1 tree, 44 leaves, m

(0.945503703866909, 0.8791810505862281)

## GradientBoostingClassifier 

HistGradientBoostingClassifier is recommended for datasets with a large number of classes. Since this is a binary classification problem, I decided to try the more established GradientBoostingClassifier class also.

In [18]:
from sklearn.ensemble import GradientBoostingClassifier 

In [117]:
GradientBoostingClassifier?

In [19]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to_nn_full.pkl').load()

xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

### Defaults

n_estimators was increased to 1000 from its default value of 100.

In [61]:
clf = GradientBoostingClassifier(loss='deviance', n_estimators=1000, verbose=1)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss   Remaining Time 
         1           0.2858            3.30m
         2           0.2787            3.31m
         3           0.2732            3.41m
         4           0.2681            3.42m
         5           0.2632            3.47m
         6           0.2587            3.47m
         7           0.2550            3.47m
         8           0.2521            3.47m
         9           0.2490            3.47m
        10           0.2456            3.46m
        20           0.2276            3.40m
        30           0.2183            3.34m
        40           0.2126            3.27m
        50           0.2085            3.23m
        60           0.2047            3.22m
        70           0.2017            3.20m
        80           0.1991            3.17m
        90           0.1969            3.15m
       100           0.1948            3.12m
       200           0.1771            2.83m
       300           0.1646            2.52m
       40

(0.9705755015972327, 0.8568501495478038)

### Hyperparameter Tuning

In [63]:
parameters1 = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [40, 50, 60, 70, 1000]
}

In [72]:
parameters2 = {
 'max_depth': [3, 6, 9, 12],
 'subsample': [0.8, 0.66],
 'min_samples_split': [2, 5, 10],
 'max_leaf_nodes': [None, 20, 31, 40],
 'min_samples_leaf': [1, 5],
 'random_state': [33],
 }

In [78]:
parameters3 = {
 'max_depth': [8, 9, 10],
 'subsample': [0.60, 0.66, 0.72],
 'min_samples_split': [3, 5, 8],
 'max_leaf_nodes': [None, 45, 50, 60],
 'min_samples_leaf': [5, 7, 9],
 'random_state': [33],
}

In [81]:
parameters4 = {
 'max_depth': [8, 9, 10],
 'subsample': [0.68, 0.72, 0.76],
 'min_samples_split': [1, 2, 3],
 'max_leaf_nodes': [50, 52, 54],
 'min_samples_leaf': [6, 7, 8],
 'random_state': [33],
 }

In [99]:
parameters5 = {
    'n_estimators': [40, 50, 60, 70, 100, 1000]
}

In [100]:
clf = GridSearchCV(GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10), 
                         parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [101]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 12s, sys: 1.32 s, total: 1min 13s
Wall time: 4min 14s


(GradientBoostingClassifier(max_depth=10, max_features='sqrt', max_leaf_nodes=52,
                            min_samples_split=6, n_estimators=1000,
                            subsample=0.72, tol=1e-08),
 0.8841218032816766,
 {'n_estimators': 1000})

In [104]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.2693           0.0407            1.12m
         2           0.2610           0.0080            1.04m
         3           0.2520           0.0075           59.71s
         4           0.2429           0.0055            1.08m
         5           0.2422           0.0028            1.06m
         6           0.2346           0.0033            1.05m
         7           0.2353           0.0050            1.04m
         8           0.2226           0.0042            1.03m
         9           0.2224           0.0027            1.03m
        10           0.2239           0.0020            1.04m
        20           0.1882           0.0006            1.07m
        30           0.1758           0.0002            1.03m
        40           0.1648          -0.0000            1.03m
        50           0.1593          -0.0001            1.03m
        60           0.1503          -0.0010            1.02m
       

(1.0, 0.8318749851413832)

In [103]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.005, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3119           0.0022            1.08m
         2           0.3162           0.0018            1.10m
         3           0.3054           0.0018            1.03m
         4           0.3067           0.0014            1.06m
         5           0.3047           0.0014            1.05m
         6           0.3057           0.0015            1.05m
         7           0.2964           0.0015            1.06m
         8           0.3050           0.0014            1.05m
         9           0.2936           0.0013            1.06m
        10           0.2941           0.0012            1.02m
        20           0.2828           0.0009           58.00s
        30           0.2784           0.0006           58.51s
        40           0.2696           0.0005           58.59s
        50           0.2593           0.0004           57.99s
        60           0.2554           0.0004           58.28s
       

(0.9152136625826836, 0.843631924082018)

In [20]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.01, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3042           0.0047            1.32m
         2           0.2985           0.0036            1.12m
         3           0.2985           0.0032            1.03m
         4           0.2927           0.0029            1.07m
         5           0.2927           0.0029            1.04m
         6           0.2857           0.0024            1.04m
         7           0.2843           0.0024            1.01m
         8           0.2843           0.0018            1.01m
         9           0.2741           0.0019            1.01m
        10           0.2770           0.0017            1.03m
        20           0.2601           0.0014           59.83s
        30           0.2482           0.0008           58.18s
        40           0.2370           0.0006           56.85s
        50           0.2368           0.0006           55.43s
        60           0.2276           0.0004           55.59s
       

(0.9530461225192698, 0.8791834180003287)

In [21]:
to = (path/'to_nn_full.pkl').load()
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [22]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

In [23]:
for cat in cats:
    to[cat] = to[cat].astype('category')

In [24]:
HistGradientBoostingClassifier?

In [25]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.242 GB of training data: 1.191 s
Binning 0.027 GB of validation data: 0.040 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.83731, val score: 0.78563, in 0.148s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87314, val score: 0.84035, in 0.192s
[3/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.87488, val score: 0.84051, in 0.205s
[4/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.88056, val score: 0.84451, in 0.175s
[5/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88272, val score: 0.84549, in 0.169s
[6/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.88514, val score: 0.84660, in 0.355s
[7/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.88573, val score: 0.84823, in 0.225s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.88528, val score: 0.84654, in 0.188s
[9/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.88609, val score: 0.84730, in 0.212s
[10/1000] 1 tree, 4

[85/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93687, val score: 0.88020, in 0.270s
[86/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.93708, val score: 0.88048, in 0.308s
[87/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.93741, val score: 0.88090, in 0.272s
[88/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93803, val score: 0.88102, in 0.274s
[89/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.93830, val score: 0.88162, in 0.396s
[90/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.93900, val score: 0.88166, in 0.547s
[91/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.93954, val score: 0.88176, in 0.484s
[92/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.94013, val score: 0.88184, in 0.481s
[93/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.94067, val score: 0.88204, in 0.742s
[94/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.94099, val score: 0.88177, in 0.277s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.96621, val score: 0.88901, in 0.385s
[170/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.96660, val score: 0.88901, in 0.343s
[171/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.96694, val score: 0.88909, in 0.337s
[172/1000] 1 tree, 44 leaves, max depth = 20, train score: 0.96710, val score: 0.88898, in 0.337s
[173/1000] 1 tree, 44 leaves, max depth = 20, train score: 0.96722, val score: 0.88907, in 0.349s
[174/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96747, val score: 0.88921, in 0.358s
[175/1000] 1 tree, 44 leaves, max depth = 21, train score: 0.96772, val score: 0.88933, in 0.364s
[176/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96794, val score: 0.88940, in 0.390s
[177/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.96818, val score: 0.88963, in 0.594s
[178/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96838, val score: 0.88971, in 0.625s
[179/1000] 1 tree, 4

(0.9641418347459628, 0.8891066033041436)